<a href="https://colab.research.google.com/github/alexis-leon-delgado/Air_traffic_data_analysis_with_R/blob/main/Air_traffic_data_analysis_with_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Definition


The aim of this work is to manipulate air transport data efficiently. The implemented features are:

*   Detect **rerouted flights**. These are those flights where the airport of arrival in the planned route is different from the airport of arrival in the actual route.
*   Analyse **delays** and deviations of non-rerouted flights, in distance and time. The **delay** of a flight is equal to the difference between the actual and the planned arrival time, as long as it is positive. If the difference is negative or zero, the delay is zero. The **distance deviation** of a flight is equal to the difference between the actual distance traveled and the planned distance, as long as it is positive. If the difference is negative or zero, the distance deviation is equal to zero. All **average values** of delay and deviation are flight averages.





The work deliverable is an R script named `XX_TA.txt`, where `XX` is the name of the group. which contains only the function `treball_ta <- function(m1, m3) {...}` which takes as input variables the names of the files `m1` and `m3` and returns a vector with ten components. These files contain a list with the `route`, `flight` and `point` tables resulting from filtering DDR2 data (*Filtering DDR2*). Within the function you will also need to read the `airports.dat` file with airport information.

The function can only use the functions:

`'´:=´' 'as.numeric' 'c' 'difftime' 'fread' 'hour' 'is.na' 'mean' 'merge' 'nrow' 'order' 'readRDS' 'return' 'rm' 'setnames' 'sum' 'unique'`

The `treball_ta` function must return a vector of ten components, in the following order:

For **rerouted flights**:

*   Component 1: Planned destination airport with the most diverted flights
*   Component 2: Country of origin airport with the most diverted flights
*   Component 3: country with the most flights diverted from their airports of origin
*   Component 4: Airline with the highest value of the quotient between diverted flights and total flights

For **non-rerouted flights** (for component 7 we will also consider the set of diverted and non-rerouted flights):

*   Component 5: origin airport of the flight with the largest distance deviation of flights with the same country of origin and destination
*   Component 6: Airline with the longest average delay
*   Component 7: origin airport with the smallest average distance deviation
*   Component 8: Expected departure time with longest average delay
*   Component 9: Arrival airport with the largest distance deviation to total planned distance ratio
*   Component 10: Aircraft model of flights with the smallest average value of the quotient between the delay and the total planned time

*Each component must have a unique value. In the event of a tie, the tie is broken by taking the first solution in alphabetical or numerical order.*

*The process of evaluating the results must be done in such a way that all flights are taken into account, even if we do not have some information about them available.*

# Code

Configuration:

In [ ]:
install.packages("data.table")

library(data.table)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



Data download:

In [ ]:
download.file("https://www.dropbox.com/s/ynta4tz4gy9mk3n/20181208_20181212_m1.RDS?dl=1", "20181208_20181212_m1.RDS")
download.file("https://www.dropbox.com/s/0gzpt98no7sq6qu/20181208_20181212_m3.RDS?dl=1", "20181208_20181212_m3.RDS")
download.file("https://www.dropbox.com/s/a4s903mpeue4v6o/20170904_20170908_m1.RDS?dl=1", "20170904_20170908_m1.RDS")
download.file("https://www.dropbox.com/s/fdsvmc5wf2m544o/20170904_20170908_m3.RDS?dl=1", "20170904_20170908_m3.RDS")

download.file("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat", "airports.dat")

#Function `treball_ta.txt`

In [ ]:
treball_ta <- function(m1, m3) {

# Data reading and results initialisation:

airports  <- fread("airports.dat")
inputM1   <- readRDS(m1)
inputM3   <- readRDS(m3)
results   <- c()

# Component 1:

Comp3_table_AUXroute  <- inputM3$route[order(fl_id, time1)][, .(fl_id, fl_des=id2)][, .SD[.N], by = "fl_id"]            # Extraction of flight id and flight destination for each flight
rerouted_fl_id        <- Comp3_table_AUXroute[!inputM3$flight[,.(fl_id, fl_des)], on=.(fl_id, fl_des)]                  # Computation of rerouted flight ids
rerouted_flight_tab   <- merge(inputM3$flight[, .(fl_id, fl_ori, fl_des, airl)],rerouted_fl_id[, .(fl_id)],by="fl_id")  # Merging of flight data with the rerouted flight ids
results[1]            <- rerouted_flight_tab[, .N, by=fl_des][order(-N,fl_des)][1,fl_des]                               # Computation of flight destination with the most rerouted flights

# Component 2:

airports_MAX <- rerouted_flight_tab[, .N, by=fl_ori][order(-N,fl_ori)][N == N[1],]                                      # Computation of flights origin with the most rerouted flights
results[2] <- merge(airports[, .(V4, V6)], airports_MAX[, .(V6 = fl_ori)], by="V6")[order(V4)][1, V4]                   # Computation of the corresponding country
                                                                 
# Component 3:

origin_count    <- rerouted_flight_tab[, .N, by=fl_ori]                                                                 # Extraction of rerouted flight origins
origin_country  <- merge(airports[, .(country=V4, fl_ori=V6)], origin_count, by="fl_ori")[, .(country, N)]              # Merging of flight origins with the airports data
sum_country     <- origin_country[, .(rerouted_flights=sum(N)),by=country]                                              # Computation of total rerouted flights per country
results[3]      <- sum_country[order(-rerouted_flights,country)][1,country]                                             # Computation of the country with the most rerouted flights

# Component 4:

rerouted_airline    <- rerouted_flight_tab[, .N, by=airl]                                                               # Extraction of number of rerouted flights per airline
total_airline       <- inputM3$flight[, .N, by=airl]                                                                    # Extraction of number of total flights per airline
flights_by_airline  <- merge(rerouted_airline[, .(airl, rerouted_N=N)], total_airline[, .(airl, total_N=N)], by="airl") # Merging of airlines with rerouted flights and the corresponding flight data
quocient_airline    <- flights_by_airline[, .(quocient_N=rerouted_N/total_N), by=airl]                                  # Computation of the quocient
results[4]          <- quocient_airline[order(-quocient_N,airl)][1,airl]                                                # Computation of the airline with the highest quocient

# Component 5:

non_rerouted_flight_tab   <- merge(inputM3$flight, Comp3_table_AUXroute, by=c("fl_id","fl_des"))                                                              # Merging of flight data, flight id and flight destination for non rerouted flights
plan_dist_tab             <- inputM1$route[,.(plan_dist=sum(dist)),by="fl_id"]                                                           # Extraction of planned distance by flight id
real_dist_tab             <- inputM3$route[,.(real_dist=sum(dist)),by="fl_id"]                                                           # Extraction of real distance by flight id
fl_distances              <- merge(plan_dist_tab, real_dist_tab, by="fl_id")                                                                                  # Merging of flight distances
fl_distances              <- fl_distances[,diff_distance := real_dist-plan_dist]                                                                              # Computation of distance difference
fl_distances              <- fl_distances[diff_distance<0, diff_distance := 0]                                                                                # Correction of negative distances to a zero value
non_rerouted_fl_distance  <- merge(fl_distances, non_rerouted_flight_tab, by="fl_id")                                                                         # Merging of flight distances with the non rerouted flights data
country_oriTab            <- merge(inputM3$flight[, .(fl_id, fl_ori, fl_des)], airports[, .(country_ori=V4, V6)], by.x="fl_ori", by.y="V6")                   # Merging of flight data with the countries of the origin airports
national_flights          <- merge(country_oriTab, airports[, .(country_des=V4, V6)], by.x="fl_des", by.y="V6")[country_ori == country_des, .(fl_id, fl_ori)] # Filtering by only national flights regarding origin and destination
distance_flights          <- merge(non_rerouted_fl_distance[, .(fl_id, diff_distance)], national_flights, by="fl_id")                                         # Computation of distance difference for national flights
results[5]                <- distance_flights[order(-diff_distance,fl_ori)][1,fl_ori]                                                                         # Computation of flight origin with the highest distance difference

# Component 6:

Comp1_table_AUXhour       <- merge(inputM1$flight[, .(fl_id, fl_des)], inputM1$route[, .(fl_id, fl_des=id2, time2)], by=c("fl_id","fl_des"))                  # Merging of planned flight data with the planned destination arrival time
Comp3_table_AUXhour       <- merge(Comp3_table_AUXroute, inputM3$route[, .(fl_id, fl_des=id2, time2)], by=c("fl_id","fl_des"))                                # Merging of flight data with the destination arrival time
Comp1_table               <- merge(Comp1_table_AUXhour[, .(fl_id, fl_des, time2)], Comp3_table_AUXhour[, .(fl_id, fl_des, time2)], by="fl_id")                # Merging of planned and real flight destination times
nonRerouted_flights_tab   <- Comp1_table[fl_des.x == fl_des.y,][, .(fl_id, plan_arrival=time2.x, real_time=time2.y)]                                          # Filtering by only non rerouted flights
nonRerouted_flights_airl  <- merge(nonRerouted_flights_tab, inputM3$flight[, .(fl_id, airl)], by="fl_id")                                                     # Merging of non rerouted flights data with airlines data
delay_tab                 <- nonRerouted_flights_airl[, delay:=difftime(real_time,plan_arrival,"sec")][, .(fl_id, airl, delay)]                               # Computation of delays
delay_tab                 <- delay_tab[delay<0, delay:=0]                                                                                                     # Correction of negative times to a zero value
meanDelay_tab             <- delay_tab[, .(mean_delay=mean(delay)), by=airl]                                                                                  # Computation of mean delays by airlines
results[6]                <- meanDelay_tab[order(-mean_delay,airl)][1,airl]                                                                                   # Computation of airline with the highest mean delay

# Component 7:

origin_airport  <- merge(fl_distances, inputM3$flight[, .(fl_id, fl_ori)], by="fl_id")                                                                        # Merging of flight distances with flight origins data
meanDist_tab    <- origin_airport[, .(mean_dist=mean(diff_distance)), by=fl_ori]                                                                              # Computation of mean distance difference in flights with same flight origin
results[7]      <- meanDist_tab[order(mean_dist,fl_ori)][1,fl_ori]                                                                                            # Computation of the origin with the minimum mean distance difference

# Component 8:

Comp1_table_AUXdeparture  <- inputM1$route[, .(fl_id, departure_time=time1)][order(fl_id, departure_time)][, .SD[1], by = "fl_id"]                            # Extraction of planned departure times for each flight id
nonRerouted_merge         <- merge(nonRerouted_flights_tab, Comp1_table_AUXdeparture[, .(fl_id, plan_departure=hour(departure_time))], by="fl_id")            # Merging of non rerouted flight data with planned departure times
delay_tab                 <- nonRerouted_merge[, delay:=difftime(real_time,plan_arrival,"sec")][, .(fl_id, plan_departure, delay)]                            # Computation of delays
delay_tab                 <- delay_tab[delay<0, delay:=0]                                                                                                     # Correction of negative times to a zero value
meanDelay_tab             <- delay_tab[, .(mean_delay=mean(delay)), by=plan_departure]                                                                        # Computation of mean delays by planned departure times
results[8]                <- meanDelay_tab[order(-mean_delay,plan_departure)][1,plan_departure]                                                               # Computation of planned departure times with the highest mean delay

# Component 9:

quocient_Table  <- non_rerouted_fl_distance[, quocient:=diff_distance/plan_dist][, .(fl_des, quocient)]                                                       # Computation of quocient
results[9]      <- quocient_Table[order(-quocient, fl_des)][1,fl_des]                                                                                         # Computation of the destination airport with the highest quocient

# Component 10:

realTime_table            <- merge(Comp1_table_AUXdeparture, nonRerouted_flights_tab, by="fl_id")                                                                                                     # Merging of non rerouted flight data with planned departure times
quocient_realTime_table   <- realTime_table[, `:=`(delay=difftime(real_time,plan_arrival,units="secs"), totalTime=difftime(plan_arrival,departure_time,units="secs"))][, .(fl_id, delay, totalTime)]  # Computation of quocient table with delay and total flight time 
quocient_Table            <- quocient_realTime_table[delay<0, delay:=0]                                                                                                                               # Correction of negative times to a zero value
result_Table              <- quocient_Table[, quocient:=as.numeric(delay)/as.numeric(totalTime)]                                                                                                      # Computation of quocient between delay and total flight time 
ac_result_table           <- merge(result_Table, inputM3$flight[, .(fl_id, ac_type)], by="fl_id")                                                                                                     # Merging of quocient with aircraft type
meanTime_tab              <- ac_result_table[, .(mean_time=mean(quocient)), by=ac_type]                                                                                                               # Computation of mean quocients by aircraft types
results[10]               <- meanTime_tab[order(mean_time,ac_type)][1,ac_type]                                                                                                                        # Computation of aircraft type with minimum mean quocient value

return(results)
}

# ***Verification of the results:***

#Inputs

In [ ]:
m1 = "20181208_20181212_m1.RDS"
m3 = "20181208_20181212_m3.RDS"

# Execution

In [ ]:
t1 <- Sys.time()
res1 <- treball_ta(m1 = "20181208_20181212_m1.RDS", m3 = "20181208_20181212_m3.RDS")
t2 <- Sys.time()

res1 # to_show

[1] "UKKK"    "Austria" "Turkey"  "PEV"     "LMML"    "AFG"     "BGSF"   
 [8] "2"       "LMML"    "B773"

Verification of the correct result: `'UKKK''Austria''Turkey''PEV''LMML''AFG''BGSF''2''LMML''B773'`

In [ ]:
t3 <- Sys.time()
res2 <- treball_ta(m1 = "20170904_20170908_m1.RDS", m3 = "20170904_20170908_m3.RDS")
t4 <- Sys.time()

res2 # to_show

[1] "LPMA"     "Portugal" "France"   "AZU"      "LPPR"     "SAW"     
 [7] "BGBW"     "0"        "LGKZ"     "AN12"

Verification of the correct result: `'LPMA''Portugal''France''AZU''LPPR''SAW''BGBW''0''LGKZ''AN12'`


Verification that the result is a vector (shall be `TRUE`):

In [ ]:
is.vector(res1)

[1] TRUE

In [ ]:
is.vector(res2)

[1] TRUE

Verification of the execution time (shall be less than 25 seconds):

In [ ]:
t2 - t1 # res1

Time difference of 6.452518 secs

In [ ]:
t4 - t3 # res2

Time difference of 7.367728 secs